In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
// Note that this must be in its own cell

<IPython.core.display.Javascript object>

In [2]:
# External imports
import ipywidgets as ipw
#from jinja2 import Environment
#from importlib_resources import files

from aiida.orm import StructureData
from aiida.orm import load_node
from aiidalab_widgets_base import StructureBrowserWidget, StructureManagerWidget, StructureUploadWidget
from aiidalab_widgets_base import SmilesWidget
from aiidalab_widgets_base import BasicStructureEditor
from aiidalab_widgets_base import WizardAppWidget
#from aiidalab_widgets_base.bug_report import install_create_github_issue_exception_handler

In [3]:
#from aiidalab_ispg.process import WorkChainSelector
from aiidalab_ispg.steps import SubmitQeAppWorkChainStep
from aiidalab_ispg.structures import StructureSelectionStep
from aiidalab_ispg.steps import ViewQeAppWorkChainStatusAndResultsStep
from aiidalab_ispg.steps import ViewSpectrumStep

# DH: Our own hacked molecule viewer
from aiidalab_ispg.widget_viewers.viewers import StructureDataViewer

# DH This is for later for importing Jinja templates and styles
#from aiidalab_ispg import static

structure_manager_widget = StructureManagerWidget(
    importers=[
        StructureUploadWidget(title="Upload file"),
        SmilesWidget(title='SMILES'),
        StructureBrowserWidget(title="AiiDA database"),
    ],
    node_class='StructureData',
    viewer=StructureDataViewer(downloadable=False,storable=False),
#    editors = [
#        BasicStructureEditor(title="Basic Editor"),
#    ],
)

structure_selection_step = StructureSelectionStep(manager=structure_manager_widget)
structure_selection_step.auto_advance = True

submit_orca_work_chain_step = SubmitQeAppWorkChainStep()
submit_orca_work_chain_step.auto_advance = True

view_orca_work_chain_status_and_results_step = ViewQeAppWorkChainStatusAndResultsStep()
view_orca_work_chain_status_and_results_step.auto_advance = True

view_spectrum_step = ViewSpectrumStep()

# Link the application steps
ipw.dlink((structure_selection_step, 'confirmed_structure'), (submit_orca_work_chain_step, 'input_structure'))
ipw.dlink((submit_orca_work_chain_step, 'process'), (view_orca_work_chain_status_and_results_step, 'process'))
ipw.dlink((view_orca_work_chain_status_and_results_step, 'process'), (view_spectrum_step, 'process'))

# Add the application steps to the application
app = WizardAppWidget(
    steps=[
        ('Select structure', structure_selection_step),
        ('Submit work chain', submit_orca_work_chain_step),
        ('Status & Results', view_orca_work_chain_status_and_results_step),
        ('Spectrum', view_spectrum_step),
    ])

# Add process selection header
#work_chain_selector = WorkChainSelector(layout=ipw.Layout(width='auto'))

# Reset all subsequent steps in case that a new structure is selected
def _observe_structure_selection(change):
    with structure_selection_step.hold_sync():
        if structure_selection_step.confirmed_structure is not None and \
                structure_selection_step.confirmed_structure != change['new']:
            app.reset()
structure_selection_step.observe(_observe_structure_selection, 'structure')

display(app)

WizardAppWidget(children=(HBox(children=(Button(description='Previous step', disabled=True, icon='step-backwar…

ORCA parameters
{'charge': 0,
 'extra_input_keywords': [],
 'input_blocks': {'scf': {'ConvForced': 'true', 'convergence': 'tight'}},
 'input_keywords': ['def2-svp', 'pbe', 'Opt', 'AnFreq'],
 'multiplicity': 1}
ProcessState.CREATED uuid: 01c93a86-7618-4160-b2d3-31a3db8ea1f3 (pk: 244) (aiida_orca.calculations.orca_main.OrcaCalculation)


In [48]:
import scipy
import numpy as np

class Spectrum(object):
    AUtoCm = 8.478354e-30
    COEFF = scipy.constants.pi * AUtoCm**2 * 1e4 /(3 * scipy.constants.hbar * scipy.constants.epsilon_0 * scipy.constants.c)
    # Transition Dipole to Osc. Strength in atomic units
    COEFF_NEW = COEFF * 3 / 2 
    #COEFF =  scipy.constants.pi * AUtoCm**2 * 1e4 * scipy.constants.hbar / (2 * scipy.constants.epsilon_0 * scipy.constants.c * scipy.constants.m_e)
    
    def __init__(self, transitions, nsample):
        # Excitation energies in eV
        self.excitation_energies = np.array([tr['energy'] for tr in transitions], dtype=float)
        # Oscillator strengths
        self.osc_strengths = np.array([tr['osc_strength'] for tr in transitions], dtype=float)
        # Number of molecular geometries sampled from ground state distribution
        self.nsample = nsample
        self.set_defaults()
    
    # TODO: These parameters should be passed as arguments to get_spectrum()
    def set_defaults(self):
        # Default grid on x-axis in eV
        self.de = 0.02
    
    # TODO: Specialize this function for Gaussian / Lorentzian broadening
    def get_spectrum(self, x_min, x_max, x_units, y_units):
        """Returns a computer spectrum as a tuple of x and y Numpy arrays"""
        
        n_points = int( (x_max-x_min)/self.de)
        x = np.arange(x_min, x_max, self.de)
        y = np.zeros(n_points)
        return x, y
    
    def get_gaussian_spectrum(self, sigma, x_units, y_units):
        """Returns Gaussian broadened spectrum"""
        # TODO: Determine x_min automatically based on transition energies
        # and x_units
        x_min = 0
        x_max = 5
        #assert x_min < x_max
        x = np.arange(x_min, x_max, self.de)
        y = np.zeros(len(x))
        #assert len(x) == len(y)
        normalization_factor = 1 / np.sqrt(2*scipy.constants.pi) / sigma / nsample
        # TODO: Support other units
        unit_factor = self.COEFF_NEW
        for exc_energy, osc_strength in zip(self.excitation_energies, self.osc_strengths):
            prefactor = normalization_factor * unit_factor * osc_strength 
            y += prefactor * np.exp(-( (x-exc_energy)**2 )/ 2 / sigma**2 )            
        return x, y
    
    def get_lorentzian_spectrum(self, tau, x_units, y_units):
        """Returns Gaussian broadened spectrum"""
        # TODO: Determine x_min automatically based on transition energies
        # and x_units
        x_min = 0
        x_max = 5
        x = np.arange(x_min, x_max, self.de)
        y = np.zeros(len(x))
        normalization_factor = tau / 2 / scipy.constants.pi / nsample
        unit_factor = self.COEFF_NEW
        for exc_energy, osc_strength in zip(self.excitation_energies, self.osc_strengths):
            prefactor = normalization_factor * unit_factor * osc_strength 
            y += prefactor / ( (x - exc_energy)**2 + (tau**2)/4 )
        return x, y

In [50]:
transition1 = {
    'energy': 1, # Excited energy in eV
    'osc_strength': '0.016510951'
}
transition2 = {
    'energy': 2.0, # Excited energy in eV
    'osc_strength': '0.0'
}
transitions = [transition1, transition2]
nsample = 1
spec = Spectrum(transitions, nsample)
x, y = spec.get_lorentzian_spectrum(0.3, 'ev', 'cross_section')

In [51]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
output_notebook()
p = figure(title="Spectrum test",  x_axis_label='E / eV',   y_axis_label='I / cm^-2 * molecule ^ -1')
p.line(x, y, legend_label='legend title', line_width=2)
p.line(x, y, line_width=2)
show(p)

Loading BokehJS ...

In [26]:
#for xi, yi in zip(x, y):
#    print(xi, yi)

In [57]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [64]:
import bqplot.pyplot as plt
import ipywidgets as widgets

def plot_spectrum(width, kernel):
    transition1 = {
        'energy': 1, # Excited energy in eV
        'osc_strength': '0.016510951'
    }
    transition2 = {
        'energy': 2.0, # Excited energy in eV
        'osc_strength': '0.05'
    }
    transitions = [transition1, transition2]
    nsample = 1
    spec = Spectrum(transitions, nsample)
    if kernel == 'lorentzian':
        x, y = spec.get_lorentzian_spectrum(width, 'ev', 'cross_section')
    elif kernel == 'gaussian':
        x, y = spec.get_gaussian_spectrum(width, 'ev', 'cross_section')
    else:
        print("ERROR")

    fig = plt.figure(title="title title")  
    plt.plot(x, y)
    plt.xlabel("Energy")
    plt.ylabel("Intensity")
    plt.show()

In [65]:
_ = widgets.interact(plot_spectrum, width=(0.05, 1.0, 0.05), kernel=['gaussian', 'lorentzian'])

interactive(children=(FloatSlider(value=0.5, description='width', max=1.0, min=0.05, step=0.05), Dropdown(desc…